In [ ]:
import gradio as gr
from rembg import remove, new_session
from PIL import Image
import numpy as np

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip("#")
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def remove_background(img, mode, color_to_remove, strength):
    if img is None:
        return None

    model_map = {
        "Portrait Mode": "u2net_human_seg",
        "General Mode": "u2net",
        "Product Mode": "isnet-general-use",
        "Anime Mode": "isnet-anime",
    }

    model_name = model_map.get(mode, "u2net")
    session = new_session(model_name)

    output = remove(
        img,
        session=session,
        alpha_matting=True,
        alpha_matting_foreground_threshold=240,
        alpha_matting_background_threshold=10,
        alpha_matting_erode_size=10,
    )

    if color_to_remove and isinstance(color_to_remove, str):
        try:
            target_rgb = np.array(hex_to_rgb(color_to_remove))
            img_np = np.array(output.convert("RGBA"))
            diff = np.abs(img_np[:, :, :3] - target_rgb)
            mask = np.all(diff < strength, axis=-1)
            img_np[mask] = [0, 0, 0, 0]
            output = Image.fromarray(img_np)
        except Exception as e:
            print("Color removal skipped", e)

    return output


with gr.Blocks(title="Background Removal") as demo:
    gr.Markdown("Min's super awesome way to remove background")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(label="Upload Image", type="pil")
            mode = gr.Radio(
                ["Portrait Mode", "General Mode", "Product Mode", "Anime Mode"],
                value="General Mode",
                label="Removal Mode",
            )
            color_picker = gr.ColorPicker(label="Remove a specific color (optional)")
            strength = gr.Slider(10, 100, value=30, step=5, label="Color Sensitivity")
            process_btn = gr.Button("Remove Background")

        with gr.Column():
            image_output = gr.Image(label="Result Preview", interactive=True)

    process_btn.click(
        fn=remove_background,
        inputs=[image_input, mode, color_picker, strength],
        outputs=image_output,
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Anaconda\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        self.scope, self.receive, self.send
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Anaconda\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Anaconda\Lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Anaconda\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Anaconda\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Anaconda\Lib\site-p